In [1]:
from typing import List, Callable


def validate_extended_matrix(matrix: List[List[float]], n: int) -> None:
    extended_length = n + 1
    for i in range(n):
        if len(matrix[i]) != extended_length:
            raise Exception(f'Incorrect matrix was passed. Expected length {extended_length} on row {i}.')


def validate_initial_vector_length(initial_vector: List[float], n: int) -> None:
    if len(initial_vector) != n:
        raise Exception(f'The vector of initial approximation should have length of {n}')


def validate_dominating_diagonal(matrix: List[List[float]]) -> bool:
    n = len(matrix)
    for i in range(n):
        s = 0
        for j in range(n):
            if i != j:
                s += abs(matrix[i][j])

        if s >= abs(matrix[i][i]):
            return False

    return True


def copy_matrix(matrix: List[List[float]]) -> List[List[float]]:
    return [[col for col in row] for row in matrix]


def print_matrix(matrix: List[List[float]]) -> None:
    for row in matrix:
        print(row)


def solve_gauss(original_matrix: List[List[float]]) -> tuple[List[float], List[List[float]]]:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)
    operative_matrix = copy_matrix(original_matrix)

    for i in range(n):
        for j in range(i + 1, n):
            row_multiplier = operative_matrix[j][i] / operative_matrix[i][i]
            for k in range(i, n + 1):
                operative_matrix[j][k] -= row_multiplier * operative_matrix[i][k]

    ans = [0 for _ in range(n)]
    for i in reversed(range(n)):
        ans_numerator = operative_matrix[i][n]
        for j in range(i + 1, n):
            ans_numerator -= operative_matrix[i][j] * ans[j]

        ans[i] = ans_numerator / operative_matrix[i][i]

    return ans, operative_matrix


def solve_gauss_jordan(original_matrix: List[List[float]]) -> tuple[List[float], List[List[float]]]:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)
    operative_matrix = copy_matrix(original_matrix)

    for i in range(n):
        for j in filter(lambda arg1: i != arg1, range(n)):
            row_multiplier = operative_matrix[j][i] / operative_matrix[i][i]
            for k in range(i, n + 1):
                operative_matrix[j][k] -= row_multiplier * operative_matrix[i][k]

    ans = [row[n] / row[i] for i, row in enumerate(operative_matrix)]
    return ans, operative_matrix


def solve_jacobi(original_matrix: List[List[float]], initial_vector: List[float], iterations: int) -> List[float] | None:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)
    validate_initial_vector_length(initial_vector, n)

    if validate_dominating_diagonal(original_matrix) is False:
        return None

    last_approx = initial_vector

    for i in range(iterations):
        current_approx = [0 for _ in range(n)]

        for j in range(n):
            for k in range(n):
                if j == k:
                    continue

                current_approx[j] -= (original_matrix[j][k] / original_matrix[j][j]) * last_approx[k]

            current_approx[j] += original_matrix[j][n] / original_matrix[j][j]

        last_approx = current_approx

    return last_approx


def solve_gauss_seidel(original_matrix: List[List[float]], initial_vector: List[float], iterations: int) -> List[float] | None:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)
    validate_initial_vector_length(initial_vector, n)

    if validate_dominating_diagonal(original_matrix) is False:
        return None

    last_approx = initial_vector

    for i in range(iterations):
        current_approx = [0 for _ in range(n)]

        for j in range(n):
            for k in range(n):
                if j == k:
                    continue

                multiplier = current_approx[k] if k < j else last_approx[k]
                current_approx[j] -= (original_matrix[j][k] / original_matrix[j][j]) * multiplier

            current_approx[j] += original_matrix[j][n] / original_matrix[j][j]

        last_approx = current_approx

    return last_approx


def wrap_iterative_method(original_matrix: List[List[float]], func: Callable[[List[List[float]], List[float], int], List[float] | None]) -> tuple[List[float] | None, List[List[float]] | None]:
    wrapped_ans = func(original_matrix, [0 for _ in original_matrix], 1_000)
    return wrapped_ans, None


def generate_hilbert_matrix(n: int) -> List[List[float]]:
    hilbert_matrix = []
    for i in range(n):
        hilbert_matrix.append([])

        for j in range(n):
            hilbert_matrix[i].append(1 / (i + j + 1))

    return hilbert_matrix


def extend_matrix(matrix: List[List[float]], extension: List[float]) -> List[List[float]]:
    if len(matrix) != len(extension):
        raise Exception("The matrix and the extension should have the same length")

    extended_matrix = copy_matrix(matrix)
    for i, extended_el in enumerate(extension):
        extended_matrix[i].append(extended_el)

    return extended_matrix


problems = [
    extend_matrix([[9, -6, -2], [-2, 8, -3], [-1, -4, 6]], [10, 0, 0]),
    extend_matrix([[-1, -1, 6], [-1, 6, -1], [6, -1, -1]], [2, 34, 12]),
    extend_matrix([[1, 4, -3], [4, 20, 14], [3, 14, 14]], [4, 20, 14]),
    extend_matrix([[4, 1, 2], [1, -3, -1], [3, 1, 5]], [6, 3, 10]),
]

hilbert_sizes = [3, 5, 10, 20]
for hs in hilbert_sizes:
    hm = extend_matrix(generate_hilbert_matrix(hs), [1 for _ in range(hs)])
    problems.append(hm)

solving_funcs = [(solve_gauss, 'Gauss'), (solve_gauss_jordan, 'Gauss-Jordan'),
                 (lambda arg1: wrap_iterative_method(arg1, solve_jacobi), 'Jacobi'),
                 (lambda arg1: wrap_iterative_method(arg1, solve_gauss_seidel), 'Gauss-Seidel')]


def print_info(ans: tuple[List[float] | None, List[List[float]] | None, str], original_matrix: List[List[float]],
               func_name: str) -> None:
    print(func_name)
    print('Original matrix:')
    print_matrix(original_matrix)
    print()

    if ans[1] is not None:
        print('Matrix after transformations:')
        print_matrix(ans[1])
        print()

    if ans[0] is None:
        print('This method is not suitable for solving this system')
    else:
        for index, x_res in enumerate(ans[0]):
            print(f'x_{index + 1} = {x_res}')

    print("--------------------------------------------------")
    print()


def solve() -> None:
    for func, func_name in solving_funcs:
        for problem_matrix in problems:
            ans = func(problem_matrix)
            print_info(ans, problem_matrix, func_name)


solve()

Gauss
Original matrix:
[9, -6, -2, 10]
[-2, 8, -3, 0]
[-1, -4, 6, 0]

Matrix after transformations:
[9, -6, -2, 10]
[0.0, 6.666666666666667, -3.4444444444444446, 2.2222222222222223]
[0.0, 0.0, 3.3666666666666663, 2.666666666666667]

x_1 = 1.7821782178217822
x_2 = 0.7425742574257427
x_3 = 0.7920792079207922
--------------------------------------------------

Gauss
Original matrix:
[-1, -1, 6, 2]
[-1, 6, -1, 34]
[6, -1, -1, 12]

Matrix after transformations:
[-1, -1, 6, 2]
[0.0, 7.0, -7.0, 32.0]
[0.0, 0.0, 28.0, 56.0]

x_1 = 3.428571428571429
x_2 = 6.571428571428571
x_3 = 2.0
--------------------------------------------------

Gauss
Original matrix:
[1, 4, -3, 4]
[4, 20, 14, 20]
[3, 14, 14, 14]

Matrix after transformations:
[1, 4, -3, 4]
[0.0, 4.0, 26.0, 4.0]
[0.0, 0.0, 10.0, 0.0]

x_1 = 0.0
x_2 = 1.0
x_3 = 0.0
--------------------------------------------------

Gauss
Original matrix:
[4, 1, 2, 6]
[1, -3, -1, 3]
[3, 1, 5, 10]

Matrix after transformations:
[4, 1, 2, 6]
[0.0, -3.25, -1.5